In [1]:
import os

#SEASONS = list(range(2023, 2025))
SEASONS = list(range(2025, 2026))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [2]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [3]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [4]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [5]:
#for season in SEASONS:
#   await scrape_season(season)

In [6]:
standings_files = os.listdir(STANDINGS_DIR)

In [7]:
async def scrape_game(standings_file):
    # Lê o arquivo HTML com a codificação padrão
    with open(standings_file, 'r', encoding='utf-8') as f:
        html = f.read()

    # Analisa o conteúdo HTML
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    # Processa cada box score
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        
        # Ignora se o arquivo já existe
        if os.path.exists(save_path):
            continue

        # Obtém o conteúdo HTML da página
        html = await get_html(url, "#content")
        if not html:
            continue

        # Salva o conteúdo com codificação UTF-8
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(html)

In [8]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

In [10]:
standings_file = "NBA_2025_games-december.html"

def processar_jogos_futuros(standings_file):
    # Lê o arquivo HTML com a codificação padrão
    with open(standings_file, 'r', encoding='utf-8') as f:
        html = f.read()

    # Analisa o conteúdo HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # Encontra todas as linhas da tabela
    rows = soup.find_all("tr")
    
    jogos_futuros = []
    
    for row in rows:
        # Procura o link de "box score"
        box_score_link = row.find("a", href=True, text="Box Score")
        
        # Se não houver "box score", é um jogo futuro
        if not box_score_link:
            cols = row.find_all("td")
            if not cols:
                continue
            
            # Extrai as informações relevantes
            jogo = {
                "data": row.find("th").text.strip() if row.find("th") else None,
                "horario": cols[0].text.strip() if len(cols) > 0 else None,
                "time_visitante": cols[1].text.strip() if len(cols) > 1 else None,
                "time_casa": cols[3].text.strip() if len(cols) > 3 else None
            }
            jogos_futuros.append(jogo)
    
    # Retorna os jogos futuros como um DataFrame
    return pd.DataFrame(jogos_futuros)

# Processa o arquivo do mês atual
jogos_futuros = processar_jogos_futuros(os.path.join(STANDINGS_DIR, standings_file))

# Exibe os jogos futuros
print(jogos_futuros)

# Salva os jogos futuros em CSV, se necessário
jogos_futuros.to_csv("jogos_futuros_dezembro.csv", index=False)

                 data horario          time_visitante              time_casa
0   Thu, Dec 19, 2024   7:00p               Utah Jazz        Detroit Pistons
1   Thu, Dec 19, 2024   7:00p   Oklahoma City Thunder          Orlando Magic
2   Thu, Dec 19, 2024   7:00p       Charlotte Hornets     Washington Wizards
3   Thu, Dec 19, 2024   7:30p           Chicago Bulls         Boston Celtics
4   Thu, Dec 19, 2024   7:30p           Brooklyn Nets        Toronto Raptors
..                ...     ...                     ...                    ...
90  Tue, Dec 31, 2024   3:00p         Milwaukee Bucks         Indiana Pacers
91  Tue, Dec 31, 2024   7:00p    Los Angeles Clippers      San Antonio Spurs
92  Tue, Dec 31, 2024   8:00p  Minnesota Timberwolves  Oklahoma City Thunder
93  Tue, Dec 31, 2024   9:00p     Cleveland Cavaliers     Los Angeles Lakers
94  Tue, Dec 31, 2024   9:00p       Memphis Grizzlies           Phoenix Suns

[95 rows x 4 columns]


C:\Users\guepp\AppData\Local\Temp\ipykernel_10996\3578683354.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  box_score_link = row.find("a", href=True, text="Box Score")
